# Brain Tumor Detection Using a Convolutional Neural Network

In [37]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import cv2
import imutils
import numpy as np
import matplotlib.pyplot as plt
import time
from os import listdir

In [36]:
def crop_brain_contour(image, plot=False):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)
    
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    
    new_image = image[extTop[1]:extBot[1], extLeft[0]:extRight[0]]
    return new_image

In [3]:
def load_data(dir_list, image_size):
    X = []
    y = []
    image_width, image_height = image_size
    
    for directory in dir_list:
        for filename in listdir(directory):
            image = cv2.imread(directory + '\\' + filename)
            image = crop_brain_contour(image, plot=False)
            image = cv2.resize(image, dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)
            image = image / 255.
            X.append(image)
            if directory[-3:] == 'yes':
                y.append([1])
            else:
                y.append([0])
                
    X = np.array(X)
    y = np.array(y)
    X, y = shuffle(X, y)
    
    print(f'Number of examples is: {len(X)}')
    print(f'X shape is: {X.shape}')
    print(f'y shape is: {y.shape}')
    
    return X, y

In [35]:
augmented_path = 'augmented data/'
augmented_yes = augmented_path + 'yes' 
augmented_no = augmented_path + 'no'

IMG_WIDTH, IMG_HEIGHT = (240, 240)

X, y = load_data([augmented_yes, augmented_no], (IMG_WIDTH, IMG_HEIGHT))

Number of examples is: 2065
X shape is: (2065, 240, 240, 3)
y shape is: (2065, 1)


In [5]:
def split_data(X, y, test_size=0.2):
    
    X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=test_size)
    X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5)
    
    return X_train, y_train, X_val, y_val, X_test, y_test

In [6]:
X_train, y_train, X_val, y_val, X_test, y_test = split_data(X, y, test_size=0.3)

In [7]:
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of development examples = " + str(X_val.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(y_train.shape))
print ("X_val (dev) shape: " + str(X_val.shape))
print ("Y_val (dev) shape: " + str(y_val.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(y_test.shape))

number of training examples = 1445
number of development examples = 310
number of test examples = 310
X_train shape: (1445, 240, 240, 3)
Y_train shape: (1445, 1)
X_val (dev) shape: (310, 240, 240, 3)
Y_val (dev) shape: (310, 1)
X_test shape: (310, 240, 240, 3)
Y_test shape: (310, 1)


In [33]:
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return f"{h}:{m}:{round(s,1)}"

# Build the model

In [32]:
def build_model(input_shape):
    
    X_input = Input(input_shape) 
    X = ZeroPadding2D((2, 2))(X_input) 
    X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X) 
    X = MaxPooling2D((4, 4), name='max_pool0')(X)  
    X = MaxPooling2D((4, 4), name='max_pool1')(X) 
    X = Flatten()(X) 
    X = Dense(1, activation='sigmoid', name='fc')(X) 
    model = Model(inputs = X_input, outputs = X, name='BrainDetectionModel')
    
    return model

In [10]:
IMG_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)

In [11]:
model = build_model(IMG_SHAPE)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [12]:
model.summary()

Model: "BrainDetectionModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 240, 240, 3)]     0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 244, 244, 3)       0         
_________________________________________________________________
conv0 (Conv2D)               (None, 238, 238, 32)      4736      
_________________________________________________________________
bn0 (BatchNormalization)     (None, 238, 238, 32)      128       
_________________________________________________________________
activation (Activation)      (None, 238, 238, 32)      0         
_________________________________________________________________
max_pool0 (MaxPooling2D)     (None, 59, 59, 32)        0         
_________________________________________________________________
max_pool1 (MaxPooling2D)     (None, 14, 14, 32)

In [13]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [31]:
filepath="cnn-parameters-improvement-{val_acc:.2f}"
checkpoint = ModelCheckpoint("models_2/{}.model".format(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max'))

## Train the model

In [15]:
start_time = time.time()

model.fit(x=X_train, y=y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val),callbacks=[checkpoint])

end_time = time.time()
execution_time = (end_time - start_time)
print(f"Elapsed time: {hms_string(execution_time)}")

Train on 1445 samples, validate on 310 samples
Epoch 1/10
1445/1445 [==============================] - 215s 149ms/sample - loss: 0.7889 - acc: 0.6118 - val_loss: 0.7076 - val_acc: 0.4645
Epoch 2/10
1445/1445 [==============================] - 241s 167ms/sample - loss: 0.4961 - acc: 0.7640 - val_loss: 0.6889 - val_acc: 0.5097
Epoch 3/10
1445/1445 [==============================] - 242s 167ms/sample - loss: 0.5166 - acc: 0.7592 - val_loss: 0.6648 - val_acc: 0.5581
Epoch 4/10
1445/1445 [==============================] - 241s 167ms/sample - loss: 0.4317 - acc: 0.8125 - val_loss: 0.7475 - val_acc: 0.5097
Epoch 5/10
1445/1445 [==============================] - 234s 162ms/sample - loss: 0.3302 - acc: 0.8533 - val_loss: 0.9574 - val_acc: 0.4871
Epoch 6/10
1445/1445 [==============================] - 194s 134ms/sample - loss: 0.3015 - acc: 0.8754 - val_loss: 0.6891 - val_acc: 0.5935
Epoch 7/10
1445/1445 [==============================] - 193s 134ms/sample - loss: 0.2973 - acc: 0.8775 - val_loss

Let's train for a few more epochs:

In [16]:
start_time = time.time()

model.fit(x=X_train, y=y_train, batch_size=32, epochs=4, validation_data=(X_val, y_val),callbacks=[checkpoint])

end_time = time.time()
execution_time = (end_time - start_time)
print(f"Elapsed time: {hms_string(execution_time)}")

Train on 1445 samples, validate on 310 samples
Epoch 1/4
1445/1445 [==============================] - 198s 137ms/sample - loss: 0.1967 - acc: 0.9308 - val_loss: 0.7143 - val_acc: 0.6581
Epoch 2/4
1445/1445 [==============================] - 199s 138ms/sample - loss: 0.2318 - acc: 0.8983 - val_loss: 0.4702 - val_acc: 0.7871
Epoch 3/4
1445/1445 [==============================] - 194s 134ms/sample - loss: 0.1870 - acc: 0.9273 - val_loss: 0.5096 - val_acc: 0.7774
Epoch 4/4
1445/1445 [==============================] - 217s 150ms/sample - loss: 0.1755 - acc: 0.9391 - val_loss: 0.5865 - val_acc: 0.7452
Elapsed time: 0:13:27.8


In [17]:
start_time = time.time()

model.fit(x=X_train, y=y_train, batch_size=32, epochs=5, validation_data=(X_val, y_val),callbacks=[checkpoint])

end_time = time.time()
execution_time = (end_time - start_time)
print(f"Elapsed time: {hms_string(execution_time)}")

Train on 1445 samples, validate on 310 samples
Epoch 1/5
1445/1445 [==============================] - 211s 146ms/sample - loss: 0.1581 - acc: 0.9384 - val_loss: 0.8055 - val_acc: 0.6742
Epoch 2/5
1445/1445 [==============================] - 218s 151ms/sample - loss: 0.1715 - acc: 0.9419 - val_loss: 0.3907 - val_acc: 0.8226
Epoch 3/5
1445/1445 [==============================] - 106s 73ms/sample - loss: 0.1598 - acc: 0.9377 - val_loss: 0.4646 - val_acc: 0.8194
Epoch 4/5
1445/1445 [==============================] - 102s 71ms/sample - loss: 0.1231 - acc: 0.9571 - val_loss: 0.3698 - val_acc: 0.8387
Epoch 5/5
1445/1445 [==============================] - 104s 72ms/sample - loss: 0.1371 - acc: 0.9481 - val_loss: 0.3570 - val_acc: 0.8484
Elapsed time: 0:12:20.5


In [18]:
start_time = time.time()

model.fit(x=X_train, y=y_train, batch_size=32, epochs=5, validation_data=(X_val, y_val),callbacks=[checkpoint])

end_time = time.time()
execution_time = (end_time - start_time)
print(f"Elapsed time: {hms_string(execution_time)}")

Train on 1445 samples, validate on 310 samples
Epoch 1/5
1445/1445 [==============================] - 110s 76ms/sample - loss: 0.1217 - acc: 0.9536 - val_loss: 0.7190 - val_acc: 0.7323
Epoch 2/5
1445/1445 [==============================] - 112s 77ms/sample - loss: 0.1158 - acc: 0.9592 - val_loss: 0.3542 - val_acc: 0.8613
Epoch 3/5
1445/1445 [==============================] - 108s 75ms/sample - loss: 0.1026 - acc: 0.9640 - val_loss: 0.4349 - val_acc: 0.7968
Epoch 4/5
1445/1445 [==============================] - 109s 76ms/sample - loss: 0.1240 - acc: 0.9536 - val_loss: 0.3905 - val_acc: 0.8323
Epoch 5/5
1445/1445 [==============================] - 110s 76ms/sample - loss: 0.0863 - acc: 0.9737 - val_loss: 0.4291 - val_acc: 0.8194
Elapsed time: 0:9:9.0


In [19]:
start_time = time.time()

model.fit(x=X_train, y=y_train, batch_size=32, epochs=6, validation_data=(X_val, y_val),callbacks = [checkpoint])

end_time = time.time()
execution_time = (end_time - start_time)
print(f"Elapsed time: {hms_string(execution_time)}")

Train on 1445 samples, validate on 310 samples
Epoch 1/6
1445/1445 [==============================] - 109s 76ms/sample - loss: 0.0799 - acc: 0.9799 - val_loss: 0.3289 - val_acc: 0.8677
Epoch 2/6
1445/1445 [==============================] - 110s 76ms/sample - loss: 0.0656 - acc: 0.9896 - val_loss: 0.4067 - val_acc: 0.8677
Epoch 3/6
1445/1445 [==============================] - 111s 77ms/sample - loss: 0.0796 - acc: 0.9806 - val_loss: 0.3512 - val_acc: 0.8742
Epoch 4/6
1445/1445 [==============================] - 109s 76ms/sample - loss: 0.0555 - acc: 0.9896 - val_loss: 0.3434 - val_acc: 0.8806
Epoch 5/6
1445/1445 [==============================] - 110s 76ms/sample - loss: 0.0522 - acc: 0.9945 - val_loss: 0.6051 - val_acc: 0.7710
Epoch 6/6
1445/1445 [==============================] - 110s 76ms/sample - loss: 0.0466 - acc: 0.9958 - val_loss: 0.3398 - val_acc: 0.8903
Elapsed time: 0:11:0.1


In [20]:
start_time = time.time()

model.fit(x=X_train, y=y_train, batch_size=32, epochs=5, validation_data=(X_val, y_val),callbacks=[checkpoint])

end_time = time.time()
execution_time = (end_time - start_time)
print(f"Elapsed time: {hms_string(execution_time)}")

Train on 1445 samples, validate on 310 samples
Epoch 1/5
1445/1445 [==============================] - 120s 83ms/sample - loss: 0.0454 - acc: 0.9965 - val_loss: 0.4930 - val_acc: 0.7968
Epoch 2/5
1445/1445 [==============================] - 106s 73ms/sample - loss: 0.0432 - acc: 0.9945 - val_loss: 0.3782 - val_acc: 0.8806
Epoch 3/5
1445/1445 [==============================] - 103s 71ms/sample - loss: 0.0461 - acc: 0.9945 - val_loss: 0.3463 - val_acc: 0.8710
Epoch 4/5
1445/1445 [==============================] - 103s 71ms/sample - loss: 0.0360 - acc: 0.9979 - val_loss: 0.3613 - val_acc: 0.8935
Epoch 5/5
1445/1445 [==============================] - 103s 71ms/sample - loss: 0.0329 - acc: 0.9979 - val_loss: 0.4530 - val_acc: 0.8258
Elapsed time: 0:8:54.9


In [21]:
start_time = time.time()

model.fit(x=X_train, y=y_train, batch_size=32, epochs=5, validation_data=(X_val, y_val),callbacks=[checkpoint])

end_time = time.time()
execution_time = (end_time - start_time)
print(f"Elapsed time: {hms_string(execution_time)}")

Train on 1445 samples, validate on 310 samples
Epoch 1/5
1445/1445 [==============================] - 103s 71ms/sample - loss: 0.0324 - acc: 0.9979 - val_loss: 0.8397 - val_acc: 0.7258
Epoch 2/5
1445/1445 [==============================] - 104s 72ms/sample - loss: 0.0607 - acc: 0.9841 - val_loss: 0.8318 - val_acc: 0.7581
Epoch 3/5
1445/1445 [==============================] - 103s 71ms/sample - loss: 0.0435 - acc: 0.9938 - val_loss: 0.3725 - val_acc: 0.8742
Epoch 4/5
1445/1445 [==============================] - 104s 72ms/sample - loss: 0.0489 - acc: 0.9903 - val_loss: 0.3753 - val_acc: 0.8516
Epoch 5/5
1445/1445 [==============================] - 110s 76ms/sample - loss: 0.0319 - acc: 0.9979 - val_loss: 0.4487 - val_acc: 0.8355
Elapsed time: 0:8:43.8


In [22]:
history = model.history.history

In [23]:
for key in history.keys():
    print(key)

loss
acc
val_loss
val_acc


### Load the best model

In [26]:
best_model = load_model(filepath='models_2/cnn-parameters-improvement-0.89.model')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [27]:
best_model.metrics_names

['loss', 'acc']

Evaluate the best model on the testing data:

In [28]:
loss, acc = best_model.evaluate(x=X_test, y=y_test)

310/310 [==============================] - 8s 26ms/sample - loss: 0.2183 - acc: 0.9129


### Accuracy of the best model on the testing data:

In [30]:
print (f"Test Loss = {loss}")
print (f"Test Accuracy = {acc*100}%")

Test Loss = 0.2183035294375112
Test Accuracy = 91.29032492637634%


In [58]:
test_img = cv2.imread("yes/Y1.jpg")
test_img = cv2.cvtColor(test_img,cv2.COLOR_BGR2RGB)
test_img = cv2.resize(test_img,(240,240))
test_img = np.array(test_img).reshape((-1,240,240,3))

prediction = model.predict(test_img)
if int(prediction[0][0])==1:
    print("Brain tumor detected.")
else:
    print("No brain tumor present.")

Brain tumor detected.
